## 1. Подготовка данных

### 1.1 Получение данных

In [10]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train

,text
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...
...,...
1813,"Спасибо,подошло по размеру.Все понравилось в п..."
1814,"доставка быстрая, до Саратова около 2 недель. ..."
1815,"на внешний вид шапка нормальная, на большой об..."
1816,За 4 месяца товар так и не дошел до покупателя.


In [11]:
with open('data/categories.txt', encoding='utf-8') as f:
    categories = ', '.join([line.strip() for line in f])
categories

'бытовая техника, обувь, одежда, посуда, текстиль, товары для детей, украшения и аксессуары, электроника, нет товара'

### 1.2 Предобработка

In [13]:
import re

def clean_text(text):
    # убираем html-теги
    text = re.sub(r'<.*?>', '', text)
    # убираем спецсимволы и переводим в нижний регистр
    text = text.replace("\r", "").replace("\n", "").replace('"', '').lower().strip()
    return text

train['text_clean'] = train['text'].apply(clean_text)
test['text_clean'] = test['text'].apply(clean_text)

In [14]:
train.head()

,text,text_clean
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...","заказали 14.10.2017 , получили 25.10.2017 на м..."
1,"футболка хорошего качества,но футболка не как ...","футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!,все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в...","рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,плохо!!!низ рваный..деньги не вернули!открыла ...


## 2. Автоматическая разметка train

### 2.1 Загрузка и подготовка LLM Mistral-7B-Instruct

In [271]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
torch.cuda.is_available()

True

In [2]:
# from huggingface_hub import login
# login(token="")

In [21]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token 

model = AutoModelForCausalLM.from_pretrained(
    model_name,  
    dtype=torch.float16
).to("cuda")
model.config.pad_token_id = tokenizer.eos_token_id

classifier = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=12
)

C:\Users\alexander\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:30<00:00, 15.18s/it]
Device set to use cuda:0


### 2.2 Тестирование LLM автолейблинга на небольшой выборке

In [55]:
categories

'бытовая техника, обувь, одежда, посуда, текстиль, товары для детей, украшения и аксессуары, электроника, нет товара'

In [79]:
%%time

prompt_template = """
You are an expert in classifying customer reviews.  
Your task: decide which product category the review is about.  
If it's about "размер", maybe it's "одежда" or "обувь".
If the review does not match any category, answer with "нет товара".
Always choose exactly ONE category from the list below:  
{categories} 

Now classify this review:  
Review: {text} 

Write a response in the format: 'Answer: [category]'
"""

SAMPLE_SIZE = 10

prompts = [prompt_template.format(categories=categories, text=text) for text in train['text_clean'].iloc[:SAMPLE_SIZE]]
results = classifier(prompts, do_sample=False)

llm_labels = [res[0]['generated_text'].split("Answer:")[-1].strip().lower() for res in results]

sample_data = pd.concat([train['text_clean'].iloc[:SAMPLE_SIZE], pd.DataFrame({'auto_label': llm_labels})], axis=1)
pd.set_option('display.max_colwidth', None)  
sample_data

CPU times: total: 42.3 s
Wall time: 33 s


,text_clean,auto_label
0,"заказали 14.10.2017 , получили 25.10.2017 на мой размер 42, широкий как мешок. надо было все таки размер s заказать. по поводу качества хороший пуховик. мех натуральный , съемный. буду продавать .",одежда
1,"футболка хорошего качества,но футболка не как для девушек и женщин,а как на мужчину. она очень свободная. на свой м, заказала л. теперь не знаю что делать,ибо она мне велика, даже моему папе она полезет.",одежда
2,все отлично!!!,нет товара
3,"рисунок не очень чёткий, а ткань прозрачная, видно нижнее бельё",текстиль
4,"плохо!!!низ рваный..деньги не вернули!открыла спор,но его почему то закрыли",нет товара
5,"обычная майка с ужасным запахом, есть косяки на белой надписи как краска",одежда
6,"все как на фото, и цвет и форма.",одежда
7,"но рост 180 по колено,качество хорошее,доставка быстрая",обувь
8,не подошло по размеру. ткань тоже не понравилась,одежда
9,"очень крутой топ,заказала 11.11 пришел 28.11 еще и курьер принес",одежда


### 2.3 Разметка по ключевым словам

In [48]:
category_keywords = {
    "бытовая техника": ["миксер", "холодильник", "пылесос", "утюг", "духовка", "чайник"],
    "обувь": ["ботинки", "кроссовки", "туфли", "сандалии", "шлепанцы"],
    "одежда": ["футболка", "джинсы", "платье", "куртка", "рубашка", "кофта", "свитер", "шорты", "юбка"],
    "посуда": ["тарелка", "кастрюля", "ложка", "стакан", "кружка", "сковорода"],
    "текстиль": ["полотенце", "постель", "подушка", "скатерть", "одеяло", "плед"],
    "товары для детей": ["игрушка", "детский", "пелёнки", "коляска", "кроватка"],
    "украшения и аксессуары": ["кольцо", "серьги", "часы", "браслет", "сумка", "шарф"],
    "электроника": ["телефон", "наушники", "зарядка", "планшет", "ноутбук", "монитор"]
}

# Возвращает категорию по ключевым словам, если совпадение найдено
def regex_label(text):
    text_lower = text
    for cat, keywords in category_keywords.items():
        for kw in keywords:
            if kw in text_lower:
                return cat
    return None  # если совпадений нет

### 2.4 Комбинация методов на всем датасете с помощью батчинга

In [113]:
%%time
from tqdm import tqdm

BATCH_SIZE = 16
all_labels = []

for i in tqdm(range(0, len(train), BATCH_SIZE)):
    batch_texts = train['text_clean'].iloc[i:i+BATCH_SIZE]
    
    batch_prompts = [prompt_template.format(categories=categories, text=text) for text in batch_texts]
    batch_results = classifier(batch_prompts, do_sample=False)
    
    batch_llm_labels = [res[0]['generated_text'].split("Answer:")[-1].strip().lower() for res in batch_results]
    
    # проверка по ключевым словам
    for text, llm_label in zip(batch_texts, batch_llm_labels):
        rule_label = regex_label(text)
        if rule_label is not None:
            all_labels.append(rule_label)
        elif llm_label in categories:
            all_labels.append(llm_label)
        else:
            all_labels.append('нет товара')

train['auto_label'] = all_labels
# Сохраняем DataFrame
train.to_csv("train_labeled.csv", index=False, encoding="utf-8-sig")

100%|█████████████████████████████████████████████████████████████████████████████| 114/114 [8:21:28<00:00, 263.93s/it]

CPU times: total: 1h 47min 18s
Wall time: 8h 21min 28s


In [35]:
train_labeled = pd.read_csv('train_labeled.csv')
train_labeled.head(50)

,text,text_clean,auto_label
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...","заказали 14.10.2017 , получили 25.10.2017 на м...",одежда
1,"футболка хорошего качества,но футболка не как ...","футболка хорошего качества,но футболка не как ...",одежда
2,Все отлично!!!,все отлично!!!,нет товара
3,"Рисунок не очень чёткий, а ткань прозрачная, в...","рисунок не очень чёткий, а ткань прозрачная, в...",текстиль
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,плохо!!!низ рваный..деньги не вернули!открыла ...,нет товара
5,"обычная майка с ужасным запахом, есть косяки н...","обычная майка с ужасным запахом, есть косяки н...",одежда
6,"Все как на фото, и цвет и форма.","все как на фото, и цвет и форма.",одежда
7,"Но рост 180 по колено,качество хорошее,доставк...","но рост 180 по колено,качество хорошее,доставк...",обувь
8,Не подошло по размеру. Ткань тоже не понравилась,не подошло по размеру. ткань тоже не понравилась,одежда
9,"очень крутой топ,заказала 11.11 пришел 28.11 е...","очень крутой топ,заказала 11.11 пришел 28.11 е...",одежда


### 2.5 Кластеризация отзывов (альтернативная разметка)

In [113]:
%%time
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

model_emb = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
train_copy = train_labeled.copy()
# Основные категории без "нет товара"
main_categories = categories.split(', ')
main_categories.remove('нет товара')

review_embeddings = model_emb.encode(train_copy["text_clean"].tolist(), show_progress_bar=True)
category_embeddings = model_emb.encode(main_categories)

NUM_CLUSTERS = len(main_categories)

# KMeans по отзывам
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=123, n_init=30)
cluster_labels = kmeans.fit_predict(review_embeddings)

# Сопоставление кластера и категории
cluster_to_category = {}
for cluster_id in range(NUM_CLUSTERS):
    cluster_center = kmeans.cluster_centers_[cluster_id].reshape(1, -1)
    sims = cosine_similarity(cluster_center, category_embeddings)[0]
    
    best_idx = np.argmax(sims)
    best_score = sims[best_idx]
    
    if best_score < 0.35:  # порог сходства
        cluster_to_category[cluster_id] = "нет товара"
    else:
        cluster_to_category[cluster_id] = main_categories[best_idx]

# Присваиваем каждому отзыву категорию
auto_labels_cluster = [cluster_to_category[cl] for cl in cluster_labels]
train_copy["cluster_label"] = auto_labels_cluster
train_copy.head(50)

Batches: 100%|█████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 58.21it/s]


CPU times: total: 17 s
Wall time: 6.07 s


,text,text_clean,auto_label,cluster_label
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\nНа мой размер 42, широкий как мешок. Надо было все таки размер S заказать. \r\nПо поводу качества хороший пуховик. \r\nМех натуральный , съемный. \r\nБуду продавать .","заказали 14.10.2017 , получили 25.10.2017 на мой размер 42, широкий как мешок. надо было все таки размер s заказать. по поводу качества хороший пуховик. мех натуральный , съемный. буду продавать .",одежда,нет товара
1,"футболка хорошего качества,но футболка не как для девушек и женщин,а как на мужчину. она очень свободная. на свой М, заказала Л. теперь не знаю что делать,ибо она мне велика, даже моему папе она полезет.","футболка хорошего качества,но футболка не как для девушек и женщин,а как на мужчину. она очень свободная. на свой м, заказала л. теперь не знаю что делать,ибо она мне велика, даже моему папе она полезет.",одежда,одежда
2,Все отлично!!!,все отлично!!!,нет товара,украшения и аксессуары
3,"Рисунок не очень чёткий, а ткань прозрачная, видно нижнее бельё","рисунок не очень чёткий, а ткань прозрачная, видно нижнее бельё",текстиль,одежда
4,"плохо!!!Низ рваный..деньги не вернули!Открыла спор,но его почему то закрыли","плохо!!!низ рваный..деньги не вернули!открыла спор,но его почему то закрыли",нет товара,посуда
5,"обычная майка с ужасным запахом, есть косяки на белой надписи как краска","обычная майка с ужасным запахом, есть косяки на белой надписи как краска",одежда,украшения и аксессуары
6,"Все как на фото, и цвет и форма.","все как на фото, и цвет и форма.",одежда,украшения и аксессуары
7,"Но рост 180 по колено,качество хорошее,доставка быстрая","но рост 180 по колено,качество хорошее,доставка быстрая",обувь,украшения и аксессуары
8,Не подошло по размеру. Ткань тоже не понравилась,не подошло по размеру. ткань тоже не понравилась,одежда,обувь
9,"очень крутой топ,заказала 11.11 пришел 28.11 еще и курьер принес","очень крутой топ,заказала 11.11 пришел 28.11 еще и курьер принес",одежда,украшения и аксессуары


Данный метод показывает плохое качество, не используем

## 3. Классификация отзывов с помощью RuBert

### 3.1 Подготовка данных

In [411]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text_clean'], padding='max_length', truncation=True, max_length=128)

def encoder(dataset):
    return dataset.map(tokenize, batched=True)

In [373]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df, val_df = train_test_split(train_labeled, test_size=0.15, random_state=123, stratify=train_labeled.auto_label)

# Преобразуем метки в числа
le = LabelEncoder()
train_df['labels'] = le.fit_transform(train_df['auto_label'])
val_df['labels'] = le.transform(val_df['auto_label'])

# Создаем Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Токенизация
train_encodings = encoder(train_dataset).remove_columns(["text", "text_clean", "auto_label", "__index_level_0__"])
val_encodings = encoder(val_dataset).remove_columns(["text", "text_clean", "auto_label", "__index_level_0__"])

train_encodings.set_format("torch")
val_encodings.set_format("torch")

Map: 100%|█████████████████████████████████████████████████████████████████| 273/273 [00:00<00:00, 15401.78 examples/s]


### 3.2 Обучение на размеченном train

####  Подбор гиперпараметров

In [420]:
from sklearn.metrics import f1_score
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, TaskType

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {"f1": f1_score(labels, preds, average="weighted")}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=12,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    weight_decay=0.005,
    fp16=True,
    lr_scheduler_type="linear",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

#### Полное fine-tuning

In [430]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories.split(', ')))

full_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

full_trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\alexander\AppData\Local\Temp\ipykernel_10844\1404792533.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  full_trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1
1,No log,1.144224,0.388131
2,No log,1.132730,0.364298
3,No log,1.131267,0.364298


TrainOutput(global_step=291, training_loss=1.147442165518954, metrics={'train_runtime': 46.6278, 'train_samples_per_second': 397.617, 'train_steps_per_second': 24.964, 'total_flos': 304899097155840.0, 'train_loss': 1.147442165518954, 'epoch': 3.0})

#### LoRA

In [446]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories.split(', ')))

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
lora_model = get_peft_model(model, lora_config)

lora_trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

lora_trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\alexander\AppData\Local\Temp\ipykernel_10844\4011101623.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.871575,0.637037
2,No log,0.712943,0.685318
3,No log,0.641320,0.694840
4,No log,0.653976,0.712421
5,No log,0.663976,0.730504
6,0.743700,0.628467,0.751118
7,0.743700,0.614502,0.756394
8,0.743700,0.697968,0.765713
9,0.743700,0.720663,0.750476
10,0.743700,0.761688,0.753653


TrainOutput(global_step=970, training_loss=0.5885710726079253, metrics={'train_runtime': 81.4729, 'train_samples_per_second': 227.56, 'train_steps_per_second': 14.287, 'total_flos': 1030409676979200.0, 'train_loss': 0.5885710726079253, 'epoch': 10.0})

### 3.3 Разметка тестовых отзывов

In [474]:
import time

test_encodings = encoder(Dataset.from_pandas(test)).remove_columns(["text", "text_clean"])

start = time.time()
preds = lora_trainer.predict(test_encodings).predictions.argmax(axis=1)
end = time.time()
print("Среднее время классификации одного отзыва:", (end - start)/len(test))

predictions = le.inverse_transform(preds)

test_labeled = pd.DataFrame({
    "text": test['text_clean'],
    "category": predictions
})

# Сохранение категорий в файл
test_labeled.category.to_csv("submission.csv", index=False)
test_labeled.head(50)

Map: 100%|███████████████████████████████████████████████████████████████| 7276/7276 [00:00<00:00, 17579.13 examples/s]


Среднее время классификации одного отзыва: 0.0023344531040915164


,text,category
0,советую продавца,нет товара
1,по вашему это платье???? это узкая кофта !!!! за такие деньги говно!,одежда
2,"жуткая синтетика. неприятная ткань. летом не поносить из-за материала. полно торчащих ниток. очень неудобная доставка. лучше бы сделали так, чтобы посылка приходила на почту.",нет товара
3,"джемперок так себе на хилую четверку,запах голой синтетики,цвет грязно синий ,на фото выглядит презентабельнее,скорее всего носить не буду",одежда
4,"обычная х/б рубашка.не плотная,просвечивает нитки не обрезаны,в магазине вряд ли бы такую купила.качество среднее.посылка долго шла по россии.пока не стирали.",одежда
5,"свитер очень колется, лично я не смогла его из-за этого носить. в остальном не плохая вещь. вязка очень тонкая, просвечивает, но за такие деньги большего и не ждала.",одежда
6,"по размеру не подошёл , хотя читала отзывы, очень много дыр на штанах, они ни к чему(((",одежда
7,"размер 5xl на российский размер 60 совсем не подошёл, хотя измеряли по таблицесами штаны вроде бы нормальныевсе прострочено, нитки не висят",одежда
8,"блузка не пришла, заказывали 31 июня.",одежда
9,товар не пришел деньги не вернули.продавец не надежный,нет товара
